In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Exporatory Data Analytics**

In [ ]:
import pandas as pd

In [ ]:
# read dataset
df = pd.read_csv('/kaggle/input/titanic/train.csv')

In [ ]:
# head of data
df.head()

In [ ]:
# tail of data
df.tail()

In [ ]:
# check information of data
df.info()

In [ ]:
# check unique value of ticket
print(len(df['Ticket'].unique()))

In [ ]:
# check unique value of cabin
print(len(df['Cabin'].unique()))

In [ ]:
# check unique value of embarked
print(len(df['Embarked'].unique()))

In [ ]:
# drop unused column
drop = ['PassengerId', 'Name', 'Ticket']
df = df.drop(columns=drop)

In [ ]:
# do encoding to convert object data type into numeric
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df['Sex'] = le.fit_transform(df['Sex'])
df['Cabin'] = le.fit_transform(df['Cabin'])
df['Embarked'] = le.fit_transform(df['Embarked'])

In [ ]:
df.info()

In [ ]:
# use knnimputer to fill missing value

import pandas as pd
from sklearn.impute import KNNImputer

import pandas as pd
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.hist(bins = 30, figsize=(20, 15), color = 'red');df = df

In [ ]:
df.corr().style.background_gradient(cmap="Blues")

# **Feature Engineering**

In [ ]:
X = df.drop(columns='Survived')
y = df['Survived']

In [ ]:
X.info()

In [ ]:
X.head()

# **Data Splitting**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=1)

# **Build The Model**

*decision tree*

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=0,
                            criterion='entropy')

In [ ]:
dtc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

# do a prediction using test data
npr_pred = dtc.predict(X_test)
acc_dtc = accuracy_score(y_test, npr_pred)
acc_dtc

In [ ]:
from sklearn.metrics import classification_report
# make a classification report
print(classification_report(y_test, npr_pred))

In [ ]:
import seaborn as sns

sns.countplot(x=y)

In [ ]:
pip install imbalanced-learn

In [ ]:
pip install --upgrade imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [ ]:
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_res,
                                                                    y_res,
                                                                    test_size=0.2,
                                                                    random_state=1)

In [ ]:
dtc_res = DecisionTreeClassifier(random_state=0,
                            criterion='entropy')

In [ ]:
dtc_res.fit(X_train_res, y_train_res)

In [ ]:
from sklearn.metrics import accuracy_score

# do a prediction using test data
ovsdtc_pred = dtc_res.predict(X_test)
acc_dtc = accuracy_score(y_test, ovsdtc_pred)
acc_dtc

*ccp decison tree*

In [ ]:
prdtc = DecisionTreeClassifier(random_state=11, 
          criterion='entropy')

In [ ]:
path = prdtc.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=11,
                                 ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
import matplotlib.pyplot as plt
train_scores = [clf.score(X_train_res, y_train_res) for clf in clfs]
test_scores = [clf.score(X_test_res, y_test_res) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
# in alpha -+0.3 accuracy model has incrased, use alpha 0.015 to pruning parameter (ccp_alpha)
# when more the alpha increases, the more the decision tree is cut
dtcpr = DecisionTreeClassifier(random_state=11,
                             ccp_alpha=0.001, 
                             criterion='entropy')
dtcpr.fit(X_train_res,y_train_res)

best model

In [ ]:
# do a prediction and check accuracy score
dtcpr_pred = dtcpr.predict(X_test)
acc_dtcpr = accuracy_score(y_test, dtcpr_pred)
acc_dtcpr

In [ ]:
# print classification report
print(classification_report(y_test, dtcpr_pred))

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# scaling
ss = StandardScaler() # Standardization
mms = MinMaxScaler() # MinMaxScaler

In [ ]:
ss_X_train = ss.fit_transform(X_train)
ss_X_test = ss.fit_transform(X_test)
mms_X_train = mms.fit_transform(X_train)
mms_X_test = mms.fit_transform(X_test)

In [ ]:
# fitting the train data into model
prdtc.fit(ss_X_train, y_train)

# do a prediction using test data
pr_pred = prdtc.predict(ss_X_test)
acc_ss_prdtc = accuracy_score(y_test, npr_pred)
print(f'accuration of Standarized Decision Tree : {acc_ss_prdtc}')

# fitting the train data into model
prdtc.fit(mms_X_train, y_train)

# do a prediction using test data
pr_pred=prdtc.predict(mms_X_test)
acc_mms_dtc = accuracy_score(y_test, npr_pred)
print(f'accuration of MinMaxScaled Decision Tree : {acc_mms_dtc}')

*bagging classifier*

In [ ]:
# declare model bagging classifier with best parameter from hyperparameter tuning
from sklearn.ensemble import BaggingClassifier
bcl = BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',random_state=11), 
                        n_estimators=50, max_samples=0.5, random_state=11, n_jobs=-1)

In [ ]:
# fitting the model
bcl.fit(X_train_res, y_train_res)

In [ ]:
# do prediction with data test
bcl_pred = bcl.predict(X_test_res)

In [ ]:
# write accuracy score
acc_bcl = accuracy_score(y_test_res, bcl_pred)
acc_bcl

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100,
                            criterion='entropy',
                            max_features=0.5,
                            min_samples_leaf=10,
                            random_state=11,
                            n_jobs=1)

In [ ]:
# fit data train into model
rf.fit(X_train_res, y_train_res)

In [ ]:
# do prediction
rf_pred = rf.predict(X_test_res)

In [ ]:
# print accuracy score
acc_rf = accuracy_score(y_test_res, rf_pred)
acc_rf

In [ ]:
# print classification report
print(classification_report(y_test_res, rf_pred))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
def model(classifier):
    classifier.fit(X_train,y_train)
    ml_pred = classifier.predict(X_test)
    cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=1)
    print(classification_report(y_test, ml_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
model(knn)

In [ ]:
from sklearn.svm import SVC
svm = SVC()
model(svm)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
model(mnb)

In [ ]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
# drop unused column

drop = ['PassengerId', 'Name', 'Ticket']
test = test.drop(columns=drop)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

test['Sex'] = le.fit_transform(test['Sex'])
test['Cabin'] = le.fit_transform(test['Cabin'])
test['Embarked'] = le.fit_transform(test['Embarked'])

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer

import pandas as pd
from sklearn.impute import KNNImputer

# Buat objek KNNImputer dengan jumlah tetangga terdekat yang akan digunakan
imputer = KNNImputer(n_neighbors=3)

# Gunakan objek KNNImputer untuk mengisi nilai yang hilang dalam dataframe
test = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)

In [ ]:
test.info()

In [ ]:
test.head()

export and prediction

In [ ]:
t = [[1.0, 0.0, 38.0, 1.0, 0.0, 71.2833, 81.0, 0.0]]

In [ ]:
import numpy as np

test_arr = np.array(test)

In [ ]:
import os
print(os.getcwd())  # Print current working directory

In [ ]:
import pickle
with open('bcl-model.pkl', 'wb') as f:
    pickle.dump(bcl, f)

In [ ]:
import pickle

# Load trained model
with open('/kaggle/working/bcl-model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
model.predict(t)

In [ ]:
result = model.predict(test_arr)

In [ ]:
print(result)

In [ ]:
results = pd.DataFrame(result)
results = results.rename(columns={0: 'Survived'})


In [ ]:
results.head()

In [ ]:
len(results)

In [ ]:
results.insert(0, 'PassengerId', range(892, 892 + len(result)))

In [ ]:
results.head()

In [ ]:
results['Survived'] = results['Survived'].astype(int)

In [ ]:
results.head()

In [ ]:
results.to_csv('bcl-pred.csv')

In [ ]:
results

In [ ]:
results.to_csv('bcl-predict.csv')

In [ ]:
# results.info()

In [ ]:
# results

In [ ]:
# results.to_csv('bcl-pred.csv', index=False)

In [ ]:
# results.to_csv("bcl-resuslts.csv", index=False, float_format='%.0f')

In [ ]:
# results_final = results['Survived']
# pd.DataFrame(results_final)

In [ ]:
# import pandas as pd

# # Assuming you have a Series named results_final
# results_final.name = 'y'

# results_final

In [ ]:
# import pandas as pd

# # Assuming you have a Series named results_final
# results_final = results_final.to_frame(name='y')

# results_final

In [ ]:
# results_final.to_csv('my_submission.csv', index=False)